LSTM model architecture. 
2025-11-01.

For the architecture:

Input x: assuming tensor of shape [B, C, T]
- B = batch size
- C = number of features per timestep (3 according to Divy)
- T = sequence length (time dim)

Output x: Tensor of shape [B, 5] 
- Probabilities for each finger (0-1)

Hyperparameters to be tuned, but current hyperparameters should be solid.

In [ ]:
class LSTM_model(nn.Module):
    def __init__(self):
        super(LSTM_model, self).__init__()
        self.lstm1 = nn.LSTM(input_size=3, hidden_size=64, batch_first=True)
        self.lstm2 = nn.LSTM(input_size=64, hidden_size=128, batch_first=True)
        self.fc1 = nn.Linear(128, 64)
        self.dropout_fc = nn.Dropout(0.5)
        self.fc2 = nn.Linear(64, 5) # 5 output for the fingers
    def forward(self, x):
        x = x.transpose(1, 2) # Transposing input: [B, C, T] → [B, T, C] for LSTM input
        x, _ = self.lstm1(x) # Forward pass 
        x, _ = self.lstm2(x) # Forward pass 
        x = x[:, -1, :]
        x = F.relu(self.fc1(x))
        x = self.dropout_fc(x)
        x = self.fc2(x)
        x = torch.sigmoid(x) # Sigmoid activation for each finger
        return x